In [3]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [4]:
# Constants
exercise = "squat"
keypoint_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [12]:
# Calculate the angle between three points given their coordinates
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [13]:
# Process the JSON files and extract pose data
def process_json_files(data_dir):
    data = []

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                for frame_data in json_data["joints3d_25"]:
                    pose_data = []

                    # Extract coordinates for each body part
                    for part_coords in frame_data:
                        pose_data.append(part_coords)

                    # Calculate angles between consecutive joints
                    angles = []
                    for i in range(len(pose_data)-2):
                        coord1 = pose_data[i]
                        coord2 = pose_data[i+1]
                        coord3 = pose_data[i+2]
                        angle = calculate_angle(coord1, coord2, coord3)
                        angles.append(angle)

                    # Append the angles to the list
                    data.append(angles)

    return data

In [14]:
# Specify the data directory
data_dir = "D:/AI/pos_estimation/train/train"

In [ ]:
# Process the JSON files and extract pose data
data = process_json_files(data_dir)

# Print the extracted angles
for angles in data:
    print(angles)

In [19]:
import numpy as np
import os
import json
import csv

# Calculate the angle between three points given their coordinates
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Process the JSON files and extract pose data
def process_json_files(data_dir):
    data = {}
    s08_angles = []

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Determine the label based on whether it's the trainer or trainee folder
        if folder_name == "s08":
            trainee = "s08"
        else:
            trainee = folder_name

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                for frame_data in json_data["joints3d_25"]:
                    pose_data = []

                    # Extract coordinates for each body part
                    for part_coords in frame_data:
                        pose_data.append(part_coords)

                    # Calculate angles between consecutive joints
                    angles = []
                    for i in range(len(pose_data) - 2):
                        coord1 = pose_data[i]
                        coord2 = pose_data[i + 1]
                        coord3 = pose_data[i + 2]
                        angle = calculate_angle(coord1, coord2, coord3)
                        angles.append(angle)

                    # Append the angles to the corresponding trainee
                    if trainee == "s08":
                        s08_angles.append(angles)
                    else:
                        if trainee not in data:
                            data[trainee] = []
                        data[trainee].append(angles)

    return data, s08_angles


# Calculate the angle difference between s08 and other trainees
def calculate_angle_difference(s08_angles, trainee_angles):
    angle_diff = []
    for s08_angle, trainee_angle in zip(s08_angles, trainee_angles):
        diff = np.abs(np.array(s08_angle) - np.array(trainee_angle))
        angle_diff.append(diff)
    return angle_diff


# Save angle differences to a CSV file
def save_angle_differences_to_csv(angle_diff, trainees, filename):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        header = ["Joint"] + trainees
        writer.writerow(header)
        for i, diff in enumerate(angle_diff):
            row = [f"Joint {i + 1}"] + list(diff)
            writer.writerow(row)


# Specify the data directory
data_dir = "D:/AI/pos_estimation/train/train"

# Process the JSON files and extract pose data
data, s08_angles = process_json_files(data_dir)

# Extract trainee names
trainees = list(data.keys())

# Calculate the angle difference between s08 and other trainees
angle_diff = calculate_angle_difference(s08_angles, list(data.values()))

# Transpose the angle differences matrix
angle_diff_transposed = np.transpose(angle_diff)

# Save angle differences to a CSV file
filename = "s08_angle_differences.csv"
save_angle_differences_to_csv(angle_diff_transposed, trainees, filename)

print(f"Angle differences saved to {filename}.")


Angle differences saved to s08_angle_differences.csv.


In [34]:
import numpy as np
import os
import json
import csv

# Calculate the angle between three points given their coordinates
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Process the JSON files and extract pose data
def process_json_files(data_dir):
    data = {}

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Determine the label based on whether it's the trainer or trainee folder
        if folder_name == "s08":
            label = "correct"
        else:
            label = "incorrect"

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                for frame_data in json_data["joints3d_25"]:
                    pose_data = []

                    # Extract coordinates for each body part
                    for part_coords in frame_data:
                        pose_data.append(part_coords)

                    # Calculate angles between consecutive joints
                    angles = []
                    for i in range(len(pose_data) - 2):
                        coord1 = pose_data[i]
                        coord2 = pose_data[i + 1]
                        coord3 = pose_data[i + 2]
                        angle = calculate_angle(coord1, coord2, coord3)
                        angles.append(angle)

                    # Append the angles to the corresponding trainee
                    if label not in data:
                        data[label] = []
                    data[label].append(angles)

    return data


# Save angles to a CSV file
def save_angles_to_csv(angles, filename):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Trainee"] + [f"Joint {i + 1}" for i in range(len(angles["correct"][0]))])  # Update this line
        for label, angle_list in angles.items():
            for angle_data in angle_list:
                writer.writerow([label] + angle_data[:len(angles["correct"][0])])  # Update this line

# Specify the data directory
data_dir = "D:/AI/pos_estimation/train/train"

# Process the JSON files and extract pose data
angles = process_json_files(data_dir)

# Save angles to a CSV file
filename = "angles.csv"
save_angles_to_csv(angles, filename)

print(f"Angles saved to {filename}.")

# Calculate MJAE and MJPE
angles_array = np.concatenate((np.array(angles["correct"]), np.array(angles["incorrect"])))
mjae = np.mean(np.abs(angles_array[1:] - angles_array[0]), axis=0)
mjpe = np.mean(np.abs(angles_array[1:] - angles_array[0]) / angles_array[0], axis=0)

print("MJAE:", mjae)
print("MJPE:", mjpe)

# Save MJAE and MJPE to a CSV file
mjae_mjpe_data = [["Joint Index", "MJAE", "MJPE"]]

for i in range(len(mjae)):
    j = i + 1
    mjae_value = mjae[i]
    mjpe_value = mjpe[i]
    mjae_mjpe_data.append([j, mjae_value, mjpe_value])

filename_mjae_mjpe = "MJAE_MJPE.csv"

with open(filename_mjae_mjpe, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(mjae_mjpe_data)

print(f"Mean Joint Angle Error (MJAE) and Mean Joint Percentage Error (MJPE) saved to {filename_mjae_mjpe}.")


Angles saved to angles.csv.
MJAE: [ 32.10912585 112.33041418  23.05595529  38.31161183 108.46673087
  16.33995247  35.62200998  36.79616985  79.19749645  39.81240957
  30.06549951  38.42706881  36.92742406  21.05978603  49.62449459
  36.90817997  21.11075582   6.5436279    8.42780321  17.07145534
  42.05574651  34.89854928  24.37325981]
MJPE: [ 0.41018526  0.63677634  0.30124747  0.56831435  0.6030853   0.32213542
  0.20624027  0.28434368 56.28911578  0.402838    0.17135523  0.33295084
  0.80242446  0.12225815  0.35914759  1.05686321  0.19696145  0.10153693
  0.09567937  0.21713071  0.39175998  0.54191764  0.23342733]
Mean Joint Angle Error (MJAE) and Mean Joint Percentage Error (MJPE) saved to MJAE_MJPE.csv.


### For test videos

In [29]:
# Define the indices of the body parts you want to detect
body_part_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# Specify the path to the test folder
test_folder = r'D:\AI\pos_estimation\test\test'

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Function to extract frames and get coordinates from a video
def extract_frames_and_get_coordinates(video_path):
    coordinates = []
    cap = cv2.VideoCapture(video_path)
    with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                frame_coordinates = []
                for idx, landmark in enumerate(landmarks):
                    if idx in body_part_indices:
                        x, y, z = landmark.x, landmark.y, landmark.z
                        frame_coordinates.append((x, y, z))
                coordinates.append(frame_coordinates)
            except:
                pass

            # Render detections
            mp.solutions.drawing_utils.draw_landmarks(
                image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS,
                mp.solutions.drawing_utils.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp.solutions.drawing_utils.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    return coordinates

# Create the main directory for saving the coordinates
output_dir = r'D:\AI\pos_estimation\test_coordinates'

# Process each test video
for video in test_videos:
    video_directory = video['directory']
    video_filename = video['filename']
    video_path = os.path.join(test_folder, video_directory, 'videos', video_filename)
    coordinates = extract_frames_and_get_coordinates(video_path)
    print("Coordinates for video {}: {}".format(video_directory, coordinates))

    # Create the output directory for the video if it doesn't exist
    video_dir = os.path.join(output_dir, 'squat', video_directory)
    os.makedirs(video_dir, exist_ok=True)

    # Save the coordinates as a JSON file
    output_file = os.path.join(video_dir, 'squat_coordinates.json')
    with open(output_file, 'w') as f:
        json.dump(coordinates, f)

    print("Saved coordinates for video {} to {}".format(video_directory, output_file))

    # Calculate angles
    angles = []
    for frame_coordinates in coordinates:
        if len(frame_coordinates) >= 3:
            a, b, c = frame_coordinates[-3:]
            angle = calculate_angle(a, b, c)
            angles.append(angle)

    # Save the angles as a CSV file in the same directory
    angles_file = os.path.join(video_dir, 'angles.csv')
    with open(angles_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Frame', 'Angle'])
        for i, angle in enumerate(angles):
            writer.writerow([i+1, angle])

    print("Saved angles for video {} to {}".format(video_directory, angles_file))
    # Save MJAE and MJPE to a CSV file
mjae_mjpe_data = [["Joint Index", "MJAE", "MJPE"]]

for i in range(len(mjae)):
    j = i + 1
    mjae_value = mjae[i]
    mjpe_value = mjpe[i]
    mjpe_mjae_data.append([j, mjae_value, mjpe_value])

filename_mjae_mjpe = "MJAE_MJPE.csv"

with open(filename_mjae_mjpe, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(mjae_mjpe_data)

print(f"Mean Joint Angle Error (MJAE) and Mean Joint Percentage Error (MJPE) saved to {filename_mjae_mjpe}.")


NameError: name 'test_videos' is not defined